بدايةً يتوجب علينا تنظيف الملف من المحارف الغير عربية والأرقام وعلامات الترقيم باستثناء(.!؟) وذلك لنتمكن من تقسيم الملف إلى جمل في المرحلة التالية. وبعد ذلك علينا إزالة كلمات التوقف والقيام بالتجذيع. من ثم نقوم بحساب تكرار كل كلمة على صعيد النص وتثقيل كل جملة من جمل النص من خلال تابع المتوسط الحسابي لكلمات الجملة .المرحلة النهائية سيتم شرحها في وقتها

In [1]:
import codecs 
import re
from nltk.tokenize import wordpunct_tokenize,sent_tokenize, WhitespaceTokenizer
import imp,os
import nltk
import math
import sys
import psycopg2
from psycopg2 import connect
from psycopg2 import extras

tp = imp.load_source('textpro', 'textpro.py')

C:\Program Files\Anaconda3\envs\python27\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


&#x202b;هنا يتم التعامل مع الملفات المشبوهة  <br />

In [2]:
# split text to sentences
def tokzr_SENT(_txt): 
    return ( re.findall(ur'(?ms)\s*(.*?(?:\.|؟|!)+)', _txt))  # split sentences


In [3]:
print "Enter the suspicious file name and path :"
file_name = raw_input("file name :  ")
path = raw_input("path :  ")


Enter the suspicious file name and path :
file name :  suspicious-document330.txt
path :  F:\ITE\.5th year\1\NLP\code fragments


In [4]:
all_suspicious_files = os.listdir("suspicious-documents")

In [5]:
list_stemmed_sus = []

In [6]:
if file_name in all_suspicious_files:    # إن كان الملف واحد من ملفات قاعدة المعطيات
    print " b"
    m = filter(None, re.split(r'(\w.+)(\.\w+)',file_name ))[0]
    
    con = None
    con = connect(dbname='plagiarisimdb2', user='postgres', host='localhost', password='ucv')
    cursor = con.cursor('cursor_unique_name', cursor_factory=psycopg2.extras.DictCursor)
    # execute the Query
    query = 'SELECT cleaned_sentence FROM public.suspicious_sentence WHERE suspicious_file_name = \''+ m + "\'"
    cursor.execute(query)
    _sentences = cursor.fetchall()
    for s in _sentences: 
        print s
        for mm in s.values():
            print tp.lightStemAr(mm)
            #tok.append(tp.lightStemAr(mm))  # stemmed sentence
    
    #...
    cursor = con.cursor()
    query = 'SELECT count(*) FROM public.suspicious_sentence WHERE suspicious_file_name = \''+ m + "\'"
    cursor.execute(query)
    file_length = cursor.fetchall()
    

else:    #إن لم يكن الملف في قاعدة المعطيات للملفات المشبوهة
    print "o"
    with codecs.open(path + "\\" + file_name ,'r',encoding = 'utf-8') as f: 
        txt = f.read()  #... 1 file
    clean_text = tp.process_text_Marwa(text=txt,removePunct=True,removeSW=True,removeNum=True)
    # string
    clean_text2 = " ".join(clean_text)
    sentences = tokzr_SENT(clean_text2)
    # stemmed sentence (and removed . ! ?)
    # sentence is string

    text_stemmed_word_tokens= []
    tokenized_sentences_length = {}  # dictionary for tokenized sentence with its length
    i =0
    file_length = len(sentences)
    for s in sentences: 
        _sentences = tp.process_text(text = s, removePunct=True,removeSW=False,removeNum=False)   # sentence without .!?
        
        stemmed_sentences = tp.lightStemAr(word_list= _sentences)  # stemmed sentence
        #tok = WhitespaceTokenizer().tokenize(_sentences) #  sentence to words
        tok = _sentences
        tokenized_sentences_length.update({i:[0,len(tok),tok]})
        i+= 1
        text_stemmed_word_tokens.extend(tok) # list of all text (stemmed)words
        list_stemmed_sus.extend(stemmed_sentences)
    #print file_length

fdist = nltk.FreqDist(text_stemmed_word_tokens)
print fdist

o
<FreqDist with 276 samples and 342 outcomes>


In [7]:

#for k in fdist: 
#    print fdist[k]

print len(text_stemmed_word_tokens)


342


من أجل كل جملة:<br/>
من اجل كل كلمة:<br/>
إحضار freqDist <br/>
جمع كل التكرارات لهذه الجملة <br/>
التقسيم على عددهم

In [8]:
q =0

while q<file_length: 
    
    _list = tokenized_sentences_length[q]  # list of 3 components
    frqDist_aSentence = [fdist[unicode(word)] for word in _list[2]]  #أخذت ليستة التوكينز ثم جبت التكرار لكل وحدة
    
    #for word in _list[2]: 
    #    print fdist[unicode(word)],word
    
    t = math.fsum(frqDist_aSentence)
    print t
    tt = t/_list[1]
    _list[0] = float("%.3f" %tt)
    print _list[0],
    q+= 1


64.0
1.939 83.0
2.184 94.0
2.41 25.0
1.25 55.0
1.774 82.0
2.733 20.0
1.25 25.0
1.042 58.0
1.758 36.0
1.565 96.0
1.745


bigram,trigram نأتي الآن لمرحلة توليد الاستعلامات: سنختار أول 3 جمل وزنها كبييير ثم نشكل من كل واحدة منها جميع التشكيلات الممكنة ل 

In [9]:
# descending sort of sentences' weight

s_list = [(tokenized_sentences_length[key][0], key) for key in tokenized_sentences_length]
s_list.sort(reverse=True)
print s_list

Max_weight_sent_list = []

# we will choose the first 3 sentences >>> they are tokenized so ill work immediately for  bi&tri -gram
Max_weight_sent_list.append(tokenized_sentences_length[s_list[0][1]][2])
Max_weight_sent_list.append(tokenized_sentences_length[s_list[1][1]][2])
Max_weight_sent_list.append(tokenized_sentences_length[s_list[2][1]][2])

print Max_weight_sent_list[0]

[(2.733, 5), (2.41, 2), (2.184, 1), (1.939, 0), (1.774, 4), (1.758, 8), (1.745, 10), (1.565, 9), (1.25, 6), (1.25, 3), (1.042, 7)]
[u'\u0648\u0623\u0634\u0627\u0631', u'\u0627\u0644\u062c\u0647\u0646\u064a', u'\u0627\u0644\u0633\u0639\u0648\u062f\u064a\u0629', u'\u0646\u062c\u062d\u062a', u'\u0639\u0628\u0631', u'\u0628\u0631\u0646\u0627\u0645\u062c', u'\u0627\u0643\u062a\u0634\u0641', u'\u0627\u0644\u0645\u0645\u0644\u0643\u0629', u'\u0627\u0633\u062a\u0642\u0637\u0627\u0628', u'\u0645\u0626\u0627\u062a', u'\u0627\u0644\u0633\u064a\u0627\u062d', u'\u0627\u0644\u064a\u0627\u0628\u0627\u0646\u064a\u064a\u0646', u'\u0633\u0646\u0648\u064a\u0627', u'\u0628\u062f\u0623', u'\u062a\u0646\u0638\u064a\u0645', u'\u0627\u0644\u0628\u0631\u0646\u0627\u0645\u062c', u'\u0645\u0646\u062a\u0635\u0641', u'\u0642\u0627\u0645\u062a', u'\u0628\u062a\u0633\u064a\u064a\u0631', u'\u0631\u062d\u0644\u0627\u062a', u'\u062e\u0627\u0635\u0629', u'\u0645\u0628\u0627\u0634\u0631\u0629', u'\u0627\u0644\u064a\u0627

In [10]:
# Generating bigrams and trigrams 

Query_list1 = []
Query_list2 = []

# fill the query list first with every sentence bigrams & trigrams
for sent in Max_weight_sent_list: 
    Query_list1.extend(list(nltk.bigrams(sent)))
    Query_list2.extend(list(nltk.trigrams(sent)))



In [11]:
Query_list3= []
Query_list3.extend(list(nltk.bigrams(Max_weight_sent_list[0])))
for i in Query_list3: 
    print i[0],i[1]#,i[2]

وأشار الجهني
الجهني السعودية
السعودية نجحت
نجحت عبر
عبر برنامج
برنامج اكتشف
اكتشف المملكة
المملكة استقطاب
استقطاب مئات
مئات السياح
السياح اليابانيين
اليابانيين سنويا
سنويا بدأ
بدأ تنظيم
تنظيم البرنامج
البرنامج منتصف
منتصف قامت
قامت بتسيير
بتسيير رحلات
رحلات خاصة
خاصة مباشرة
مباشرة اليابان
اليابان المملكة
المملكة العربية
العربية السعودية
السعودية نقلت
نقلت عبرها
عبرها سائحا
سائحا يابانيا


# Indexing

In [12]:
from whoosh.fields import Schema, TEXT, STORED

schema = Schema(title=TEXT(stored=True), content=TEXT)

معلومات مهمة

whoosh.fields.STORED
This field is stored with the document, but not indexed. This field type is not indexed and not searchable. This is useful for document information you want to display to the user in the search results.

whoosh.fields.TEXT
This type is for body text. It indexes (and optionally stores) the text and stores term positions to allow phrase searching.

In [13]:
import os.path
import codecs
import re
from whoosh.index import create_in

In [14]:
if not os.path.exists("index"):
    os.mkdir("index")
ix = create_in("index", schema)

بعد إنشاء الفهرس أصبح بإمكاني فتحه :

In [15]:
from whoosh.index import open_dir

ix = open_dir("index")

********************************

يساعدني كاتب الفهرس على إضافة مستندات جديدة ويجب أن نعلم أن إضافة المستندات للفهرس هي عملية يدوية بحتةةةةةة

In [16]:
writer = ix.writer()

In [17]:
source_folder = 'source-documents'
all_source_files = os.listdir(source_folder)
for file_namee in all_source_files:
    with codecs.open(source_folder + "\\" + file_namee,'r',encoding = 'utf-8') as f: 
        txt = f.read() 
        tit = filter(None, re.split(r'(\w.+)(\.\w+)',file_namee ))[0]
        writer.add_document(title=unicode(tit), content=unicode(txt))
        
writer.commit()

In [18]:
from whoosh.qparser import QueryParser
parser = QueryParser("content", ix.schema)
#query = u"الخطوط الجوية العربية"

In [19]:
# Parse a query string
'''
from nltk import ngrams
n = 5
news = " ".join(Max_weight_sent_list[0])
fourth_grams = ngrams(news.split(), n)
'''

result = set()
for gram in Query_list3: 
    query = " ".join(gram)   
    #print "1"
    myquery = parser.parse(query)
    with ix.searcher() as searcher:
        # Use terms=True to record term matches for each hit
        results = searcher.search(myquery, terms=True,limit = 50)
        for hit in results: 
            #print hit.fields()
            result.add(hit.fields()["title"])

In [20]:
''''print "Opening the file..."            
with open("candidate files.txt", 'w' )as target: 
    for m in result: 
        target.write(m)
        target.write("\n")
    target.write(path + "\\" + file_name)
print "fine"

with open("candidate files",'r') as f:
    for line in f: 
        print line
'''

'\'print "Opening the file..."            \nwith open("candidate files.txt", \'w\' )as target: \n    for m in result: \n        target.write(m)\n        target.write("\n")\n    target.write(path + "\\" + file_name)\nprint "fine"\n\nwith open("candidate files",\'r\') as f:\n    for line in f: \n        print line\n'

In [21]:
value = u"source-document00354" 
if value in result: 
    print "t"
value = u"source-document00362" 
if value in result: 
    print "ttt"

t
ttt


In [ ]:
import sys
import psycopg2
from psycopg2 import connect
from psycopg2 import extras

from scipy import linalg, mat, dot
import numpy as np

import collections
con = None
con = connect(dbname="plagiarisimdb2", user='postgres', host='localhost', password='ucv')
#cursor = con.cursor('cursor_unique_name', cursor_factory=psycopg2.extras.DictCursor)

for i in result: 
# execute the Query
    cursor = con.cursor()
    query = 'SELECT stemmed_sentence FROM public.source_sentence WHERE source_file_name = \''+ i + "\'"
    cursor.execute(query)
    _sentences = cursor.fetchall()
    # forming trigrams
    for file1_sent in list_stemmed_sus: 
        sent1_trigram = list(nltk.trigrams(file1_sent))
        sent1_counter=collections.Counter(sent1_trigram)_
        trigram_vector1 = {}     # vector of all trigrams frequncy in sentence 1
        for file2_sent in _sentences: 
            sent2_trigram = list(nltk.trigrams(file2_sent))
            sent2_counter=collections.Counter(sent2_trigram)
            all
            all_trigram_vector2 = {}     # vector of all trigrams frequncy in sentence 2

            all_trigram_vector1 = dict(sent1_counter.items() + sent2_counter.items() +
                          [(k, sent1_counter[k] + sent2_counter[k]) for k in set(sent2_counter) & set(sent1_counter)] + [(k, 0) for k in set(sent2_counter) if k not in set(sent1_counter) ])

            all_trigram_vector2 = dict(sent1_counter.items() + sent2_counter.items() +
                          [(k, sent1_counter[k] + sent2_counter[k]) for k in set(sent2_counter) & set(sent1_counter)] + [(k, 0) for k in set(sent1_counter) if k not in set(sent2_counter)])
            
            # creating trigram vector for each sentence:
            matrix = mat( [all_trigram_vector1.values(),all_trigram_vector2.values()] )
            cosine_similarity =  dot(matrix[0],matrix[1].T)/np.linalg.norm(matrix[0])/np.linalg.norm(matrix[1])
            cc=0
            count =0
            boole = False
            
            if (cosine_similarity > 0.25): 
                for cc in all_trigram_vector1: 
                    if (all_trigram_vector1[cc] != 0 & all_trigram_vector2[cc] !=0): 
                        count+= 1
                    if(count == 3): 
                        boole = True
                        break
            if boole : 
                print " 2 similar sentences"
                print file1_sent
                print file2_sent
                print "#####################"

    

In [125]:
print len(_sentences)

54
